<a href="https://colab.research.google.com/github/shubs202k/Hyperspectral-Image-Classification-using-Deep-Learning/blob/master/pretrain_on_indian_pines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up google colab environment

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification')

In [ ]:
from sample_extraction_V2_utils import *
import scipy.io as sio

## Load Indian Pines Dataset

In [ ]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [ ]:
data = uIndianPines['indian_pines_corrected']
ground_truth = gt_IndianPines['indian_pines_gt']

In [ ]:
data.shape

(145, 145, 200)

In [ ]:
ground_truth.shape

(145, 145)

## Distrubution of samples for each class

In [ ]:
class_distribution = pd.DataFrame(np.unique(ground_truth, return_counts = True))
class_distribution = class_distribution.transpose()
class_distribution.columns = ['class','samples']
class_distribution

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [ ]:
classes , counts = np.unique(ground_truth, return_counts = True)
classes = classes[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with overlap ratio 1 and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [ ]:
accuracies, overlap, train_test_split, training_samples, test_samples = pretrain_source_models(percentages = [50,60,70,80],
                                                                        classes = classes,
                                                                        cube_size = 20,
                                                                        overlap_ratios = [0.8,0.9,1],
                                                                        data = data,
                                                                        ground_truth = ground_truth,
                                                                        batch_size = 20,
                                                                        channels = 103,
                                                                        epochs = 100,
                                                                        Verbosity = 1,
                                                                        accuracies = [],
                                                                        learning_rate = 0.0001,
                                                                        source_dataset = 'indian_pines')

Streaming output truncated to the last 5000 lines.

Epoch 00010: val_categorical_accuracy improved from 0.57238 to 0.58410, saving model to /content/drive/My Drive/Hyperspectral_Image_Classification/V2//Trained_models//full_models/indian_pines_as_source_with_overlap_ratio_0.9_and_50_samples_from_each_class_in_training_set.h5
Epoch 11/100
98/98 [==============================] - 2s 17ms/step - loss: 3.4556 - categorical_accuracy: 0.6190 - val_loss: 3.4843 - val_categorical_accuracy: 0.6091

Epoch 00011: val_categorical_accuracy improved from 0.58410 to 0.60907, saving model to /content/drive/My Drive/Hyperspectral_Image_Classification/V2//Trained_models//full_models/indian_pines_as_source_with_overlap_ratio_0.9_and_50_samples_from_each_class_in_training_set.h5
Epoch 12/100
98/98 [==============================] - 2s 18ms/step - loss: 3.3876 - categorical_accuracy: 0.6534 - val_loss: 3.4316 - val_categorical_accuracy: 0.6091

Epoch 00012: val_categorical_accuracy did not improve from 0.6

In [20]:
pretrain_results = pd.DataFrame(list(zip(overlap, training_samples, test_samples, train_test_split, accuracies)))
pretrain_results.columns = ['Overlap_ratio', 'Training Samples', 'Test Samples', 'Training_Test_Split','Test_Accuracies']
pretrain_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,397,403,50,68.24
1,0.8,477,323,60,69.35
2,0.8,555,245,70,70.20
3,0.8,636,164,80,77.44
4,0.9,1956,1962,50,74.97
5,0.9,2348,1570,60,70.64
6,0.9,2740,1178,70,75.38
7,0.9,3132,786,80,74.43
8,1.0,3951,3955,50,73.10
9,1.0,4740,3166,60,69.14
